In [1]:
!pip install flask-ngrok

In [2]:
import requests
from io import BytesIO
import urllib
import pandas
import pickle
import ast

In [3]:
count_matrix="https://github.com/pavanpera/TacoChacos/blob/main/count_matrix.pickel?raw=true"
indices="https://github.com/pavanpera/TacoChacos/blob/main/indices.pickel?raw=true"
count_matrix = BytesIO(requests.get(count_matrix).content)
indices = BytesIO(requests.get(indices).content)
count_matrix = pickle.load(count_matrix)
indices = pickle.load(indices)

In [8]:
def recommendations(mov,indices,count_matrix):
    recommended_movies = pd.DataFrame()
    mov=mov.lower()
    mov=mov.strip('\'')
    # gettin the index of the movie that matches the title
    if len(indices[indices == mov])!=0:
     idx = indices[indices == mov].index[0]
     cosine_sim = cosine_similarity(count_matrix, count_matrix)
    # creating a Series with the similarity scores in descending order
     score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)
    #print(score_series)
    # getting the indexes of the 10 most similar movies
     top_10_indexes = list(score_series.index)
     count=0
    # populating the list with the titles of the best 10 matching movies
     for i in top_10_indexes:
      if count==10:
         break
      else:
        recommended_movies=recommended_movies.append([[indices[i],score_series.loc[i]]])
        count+=1
     return recommended_movies
    else:
      return "No Enough data"

In [15]:
from sklearn.metrics.pairwise import cosine_similarity
from io import BytesIO
import pickle
import requests
import pandas as pd
from flask import Flask, request
from flask_ngrok import run_with_ngrok
app = Flask(__name__)
run_with_ngrok(app)

# Set up the main route
@app.route('/')
def main():
        return "<h1>Testing my flask in Colab</h1>"
@app.route('/predict')         
def predict():
    title=request.args.get('movie_name')
    result_final = recommendations(title,indices,count_matrix)
    print(type(result_final))
    if type(result_final)!=str:
    #print(result_final)
     result_final = result_final[0][0:10].tolist()
     names = []
     for i in range(len(result_final)):
          names.append(result_final[i])
     str_names=','.join(names)
     return(str_names)
    else:
      return result_final
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://353b-35-237-229-113.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [20/Sep/2021 08:56:26] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [20/Sep/2021 08:56:28] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [20/Sep/2021 08:56:28] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [20/Sep/2021 08:56:46] "GET /predict?movie_name=%27prema%27 HTTP/1.1" 200 -


<class 'str'>


127.0.0.1 - - [20/Sep/2021 08:56:46] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [20/Sep/2021 08:57:12] "GET /predict?movie_name=%27Interstellar%27 HTTP/1.1" 200 -


<class 'pandas.core.frame.DataFrame'>


127.0.0.1 - - [20/Sep/2021 08:57:16] "GET /predict?movie_name=%27Interstellar%27 HTTP/1.1" 200 -


<class 'pandas.core.frame.DataFrame'>


http://cf44ca67e205.ngrok.io/predict?movie_name='Interstellar'